In [1]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import skew, kurtosis, norm
#from sklearn.cluster import DBSCAN, HDBSCAN
#from sklearn.preprocessing import quantile_transform
import blimpy as bl
import setigen as stg
from astropy import units as u
import psutil
%matplotlib inline

In [ ]:
csv_file = '/home/cgchoza/galaxies/complete_cadences_catalog.csv'

df = pd.read_csv(csv_file)
dfl = df.iloc[np.where(df['Band'].values == 'L')[0]]

del df

In [ ]:
h5_list = ['/datag/pipeline/AGBT19B_999_121/blc72_blp02/blc72_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5',
       '/datag/pipeline/AGBT19B_999_121/blc72_blp02/blc72_guppi_58832_16530_HIP2792_0058.gpuspec.0000.h5',
       '/datag/pipeline/AGBT19B_999_121/blc72_blp02/blc72_guppi_58832_16851_MESSIER031_0059.gpuspec.0000.h5',
       '/datag/pipeline/AGBT19B_999_121/blc72_blp02/blc72_guppi_58832_17168_HIP3077_0060.gpuspec.0000.h5',
       '/datag/pipeline/AGBT19B_999_121/blc72_blp02/blc72_guppi_58832_17485_MESSIER031_0061.gpuspec.0000.h5',
       '/datag/pipeline/AGBT19B_999_121/blc72_blp02/blc72_guppi_58832_17801_HIP3223_0062.gpuspec.0000.h5']

print(f'Starting blc72 injection ...')
print(f'Reading Waterfalls ...')
#wf = [bl.Waterfall(h5_list[i], f_start=1350-0.0005, f_stop=1350+0.0005) for i in range(6)]
wf = [bl.Waterfall(h5_list[i]) for i in range(6)]

print('Beginning Frame construction ...')

frame_list=[stg.Frame(waterfall=wf[0]),
            stg.Frame(waterfall=wf[1]),
            stg.Frame(waterfall=wf[2]),
            stg.Frame(waterfall=wf[3]),
            stg.Frame(waterfall=wf[4]),
            stg.Frame(waterfall=wf[5])]

print('Converting to Cadence ...')

c = stg.Cadence(frame_list)

center_frequencies = np.random.uniform(1690, 1875, 5)
drift_rates = np.random.uniform(-4, 4, 5)
snrs = np.random.uniform(50, 1000, 5)

for i in range(5):
    print(f'Adding signal {i} ...')
    c.add_signal(stg.constant_path(f_start=(center_frequencies[i])*u.MHz,
                               drift_rate=drift_rates[i]*u.Hz/u.s),
                           stg.constant_t_profile(level=c[0].get_intensity(snr=snrs[i])),
                           stg.sinc2_f_profile(width=2*c[0].df*u.Hz),
                           stg.constant_bp_profile(level=1),
                           doppler_smearing=True,
                           smearing_subsamples=15)

c[0].save_h5(filename='/datax/scratch/benjb/injected_blc72_guppi_58832_16209_MESSIER031_0057.h5')
c[2].save_h5(filename='/datax/scratch/benjb/injected_blc72_guppi_58832_16851_MESSIER031_0059.h5')
c[4].save_h5(filename='/datax/scratch/benjb/injected_blc72_guppi_58832_17485_MESSIER031_0061.h5')
np.save('/datax/scratch/benjb/injections_freq_DR_snr_MESSIER031_blc72.npy', np.array([center_frequencies, drift_rates, snrs]))

In [ ]:
h5_list = ['/datag/pipeline/AGBT19B_999_121/blc73_blp03/blc73_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5',
       '/datag/pipeline/AGBT19B_999_121/blc73_blp03/blc73_guppi_58832_16530_HIP2792_0058.gpuspec.0000.h5',
       '/datag/pipeline/AGBT19B_999_121/blc73_blp03/blc73_guppi_58832_16851_MESSIER031_0059.gpuspec.0000.h5',
       '/datag/pipeline/AGBT19B_999_121/blc73_blp03/blc73_guppi_58832_17168_HIP3077_0060.gpuspec.0000.h5',
       '/datag/pipeline/AGBT19B_999_121/blc73_blp03/blc73_guppi_58832_17485_MESSIER031_0061.gpuspec.0000.h5',
       '/datag/pipeline/AGBT19B_999_121/blc73_blp03/blc73_guppi_58832_17801_HIP3223_0062.gpuspec.0000.h5']

print(f'Starting blc73 injection ...')
print(f'Reading Waterfalls ...')
#wf = [bl.Waterfall(h5_list[i], f_start=1350-0.0005, f_stop=1350+0.0005) for i in range(6)]
wf = [bl.Waterfall(h5_list[i]) for i in range(6)]

print('Beginning Frame construction ...')

frame_list=[stg.Frame(waterfall=wf[0]),
            stg.Frame(waterfall=wf[1]),
            stg.Frame(waterfall=wf[2]),
            stg.Frame(waterfall=wf[3]),
            stg.Frame(waterfall=wf[4]),
            stg.Frame(waterfall=wf[5])]

print('Converting to Cadence ...')

c = stg.Cadence(frame_list)

center_frequencies = np.random.uniform(1502, 1688, 5)
drift_rates = np.random.uniform(-4, 4, 5)
snrs = np.random.uniform(50, 1000, 5)

for i in range(5):
    print(f'Adding signal {i} ...')
    c.add_signal(stg.constant_path(f_start=(center_frequencies[i])*u.MHz,
                               drift_rate=drift_rates[i]*u.Hz/u.s),
                           stg.constant_t_profile(level=c[0].get_intensity(snr=snrs[i])),
                           stg.sinc2_f_profile(width=2*c[0].df*u.Hz),
                           stg.constant_bp_profile(level=1),
                           doppler_smearing=True,
                           smearing_subsamples=15)

c[0].save_h5(filename='/datax/scratch/benjb/injected_blc73_guppi_58832_16209_MESSIER031_0057.h5')
c[2].save_h5(filename='/datax/scratch/benjb/injected_blc73_guppi_58832_16851_MESSIER031_0059.h5')
c[4].save_h5(filename='/datax/scratch/benjb/injected_blc73_guppi_58832_17485_MESSIER031_0061.h5')
np.save('/datax/scratch/benjb/injections_freq_DR_snr_MESSIER031_blc73.npy', np.array([center_frequencies, drift_rates, snrs]))

In [ ]:
h5_list = ['/datag/pipeline/AGBT19B_999_121/blc74_blp04/blc74_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5',
       '/datag/pipeline/AGBT19B_999_121/blc74_blp04/blc74_guppi_58832_16530_HIP2792_0058.gpuspec.0000.h5',
       '/datag/pipeline/AGBT19B_999_121/blc74_blp04/blc74_guppi_58832_16851_MESSIER031_0059.gpuspec.0000.h5',
       '/datag/pipeline/AGBT19B_999_121/blc74_blp04/blc74_guppi_58832_17168_HIP3077_0060.gpuspec.0000.h5',
       '/datag/pipeline/AGBT19B_999_121/blc74_blp04/blc74_guppi_58832_17485_MESSIER031_0061.gpuspec.0000.h5',
       '/datag/pipeline/AGBT19B_999_121/blc74_blp04/blc74_guppi_58832_17801_HIP3223_0062.gpuspec.0000.h5']

print(f'Starting blc74 injection ...')
print(f'Reading Waterfalls ...')
#wf = [bl.Waterfall(h5_list[i], f_start=1350-0.0005, f_stop=1350+0.0005) for i in range(6)]
wf = [bl.Waterfall(h5_list[i]) for i in range(6)]

print('Beginning Frame construction ...')

frame_list=[stg.Frame(waterfall=wf[0]),
            stg.Frame(waterfall=wf[1]),
            stg.Frame(waterfall=wf[2]),
            stg.Frame(waterfall=wf[3]),
            stg.Frame(waterfall=wf[4]),
            stg.Frame(waterfall=wf[5])]

print('Converting to Cadence ...')

c = stg.Cadence(frame_list)

center_frequencies = np.random.uniform(1350, 1500, 5)
drift_rates = np.random.uniform(-4, 4, 5)
snrs = np.random.uniform(50, 1000, 5)

for i in range(5):
    print(f'Adding signal {i} ...')
    c.add_signal(stg.constant_path(f_start=(center_frequencies[i])*u.MHz,
                               drift_rate=drift_rates[i]*u.Hz/u.s),
                           stg.constant_t_profile(level=c[0].get_intensity(snr=snrs[i])),
                           stg.sinc2_f_profile(width=2*c[0].df*u.Hz),
                           stg.constant_bp_profile(level=1),
                           doppler_smearing=True,
                           smearing_subsamples=15)

c[0].save_h5(filename='/datax/scratch/benjb/injected_blc74_guppi_58832_16209_MESSIER031_0057.h5')
c[2].save_h5(filename='/datax/scratch/benjb/injected_blc74_guppi_58832_16851_MESSIER031_0059.h5')
c[4].save_h5(filename='/datax/scratch/benjb/injected_blc74_guppi_58832_17485_MESSIER031_0061.h5')
np.save('/datax/scratch/benjb/injections_freq_DR_snr_MESSIER031_blc74.npy', np.array([center_frequencies, drift_rates, snrs]))

In [ ]:
h5_list = ['/datag/pipeline/AGBT19B_999_121/blc75_blp05/blc75_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5',
       '/datag/pipeline/AGBT19B_999_121/blc75_blp05/blc75_guppi_58832_16530_HIP2792_0058.gpuspec.0000.h5',
       '/datag/pipeline/AGBT19B_999_121/blc75_blp05/blc75_guppi_58832_16851_MESSIER031_0059.gpuspec.0000.h5',
       '/datag/pipeline/AGBT19B_999_121/blc75_blp05/blc75_guppi_58832_17168_HIP3077_0060.gpuspec.0000.h5',
       '/datag/pipeline/AGBT19B_999_121/blc75_blp05/blc75_guppi_58832_17485_MESSIER031_0061.gpuspec.0000.h5',
       '/datag/pipeline/AGBT19B_999_121/blc75_blp05/blc75_guppi_58832_17801_HIP3223_0062.gpuspec.0000.h5']

print(f'Starting blc75 injection ...')
print(f'Reading Waterfalls ...')
#wf = [bl.Waterfall(h5_list[i], f_start=1350-0.0005, f_stop=1350+0.0005) for i in range(6)]
wf = [bl.Waterfall(h5_list[i]) for i in range(6)]

print('Beginning Frame construction ...')

frame_list=[stg.Frame(waterfall=wf[0]),
            stg.Frame(waterfall=wf[1]),
            stg.Frame(waterfall=wf[2]),
            stg.Frame(waterfall=wf[3]),
            stg.Frame(waterfall=wf[4]),
            stg.Frame(waterfall=wf[5])]

print('Converting to Cadence ...')

c = stg.Cadence(frame_list)

center_frequencies = np.random.uniform(1127, 1199, 5)
drift_rates = np.random.uniform(-4, 4, 5)
snrs = np.random.uniform(50, 1000, 5)

for i in range(5):
    print(f'Adding signal {i} ...')
    c.add_signal(stg.constant_path(f_start=(center_frequencies[i])*u.MHz,
                               drift_rate=drift_rates[i]*u.Hz/u.s),
                           stg.constant_t_profile(level=c[0].get_intensity(snr=snrs[i])),
                           stg.sinc2_f_profile(width=2*c[0].df*u.Hz),
                           stg.constant_bp_profile(level=1),
                           doppler_smearing=True,
                           smearing_subsamples=15)

c[0].save_h5(filename='/datax/scratch/benjb/injected_blc75_guppi_58832_16209_MESSIER031_0057.h5')
c[2].save_h5(filename='/datax/scratch/benjb/injected_blc75_guppi_58832_16851_MESSIER031_0059.h5')
c[4].save_h5(filename='/datax/scratch/benjb/injected_blc75_guppi_58832_17485_MESSIER031_0061.h5')
np.save('/datax/scratch/benjb/injections_freq_DR_snr_MESSIER031_blc75.npy', np.array([center_frequencies, drift_rates, snrs]))